In [23]:
import db_connect
import pandas
import matplotlib.pyplot as plt   # 資料視覺化套件
import matplotlib

In [24]:
# 建立與MySQL的連線
my_connt_obj = db_connect.mysql_connect()
conn = my_connt_obj.connect()

In [25]:
# 取得三大法人期選的資料
df = pandas.read_sql("SELECT * FROM LegalDailyFutureOption WHERE TradeDate >'2021-01-01'", con=conn)

In [26]:
# 關閉連線
conn.close() 

In [27]:
# 觀察資料
df.tail()

,ID,TradeDate,TradeGroup,FutureLongQty,OptionLongQty,FutureLongAmount,OptionLongAmount,FutureShortQty,OptionShortQty,FutureShortAmount,...,OptionLongOIAmount,FutureShortOIQty,OptionShortOIQty,FutureShortOIAmount,OptionShortOIAmount,FutureOINetQty,OptionOINetQty,FutureOINetAmount,OptionOINetAmount,CreateTime
511,8126,2021-09-14,投信,18842,1,54056912,11,18623,1,53630444,...,27,19887,0,61188652,0,732,61,-2457964,27,2021-09-15 22:15:13
512,8127,2021-09-14,外資及陸資,350702,135365,396477436,513101,347718,141283,393649724,...,558354,361277,108925,235341299,974577,-243636,-38531,-114184159,-416223,2021-09-15 22:15:13
513,8131,2021-09-16,自營商,22897,141219,34517160,458717,24348,147532,32594020,...,556122,58559,90646,32132150,473098,34285,2232,51029866,83024,2021-09-16 22:59:03
514,8132,2021-09-16,投信,4,0,13800,0,317,0,1066713,...,1024,20320,0,62119080,0,-629,58,-7197187,1024,2021-09-16 22:59:03
515,8133,2021-09-16,外資及陸資,209089,67311,233130076,355387,225788,73716,247655200,...,312185,311057,52047,206501603,401438,-243342,-24488,-122866864,-89253,2021-09-16 22:59:03


# 外資

首先製作外資的：期貨留倉為多單還是空單、留倉數量。

In [28]:
# 外資 資料
df_investment = df.loc[df['TradeGroup'] == "外資及陸資"]
df_investment.tail()

,ID,TradeDate,TradeGroup,FutureLongQty,OptionLongQty,FutureLongAmount,OptionLongAmount,FutureShortQty,OptionShortQty,FutureShortAmount,...,OptionLongOIAmount,FutureShortOIQty,OptionShortOIQty,FutureShortOIAmount,OptionShortOIAmount,FutureOINetQty,OptionOINetQty,FutureOINetAmount,OptionOINetAmount,CreateTime
503,5637,2021-09-09,外資及陸資,368279,122581,439406946,691645,365721,117327,435781817,...,500484,337133,92034,212152676,800573,-228360,-29810,-98677098,-300089,2021-09-11 23:03:29
506,5640,2021-09-15,外資及陸資,296414,226722,350815494,616017,297636,228240,342518788,...,261443,288560,40418,194794987,327938,-226640,-18083,-108949424,-66495,2021-09-15 22:12:58
509,8124,2021-09-13,外資及陸資,364905,104806,388702710,476679,375575,102980,398161026,...,550270,361532,101013,232522014,948066,-246599,-32613,-117345090,-397796,2021-09-15 22:15:13
512,8127,2021-09-14,外資及陸資,350702,135365,396477436,513101,347718,141283,393649724,...,558354,361277,108925,235341299,974577,-243636,-38531,-114184159,-416223,2021-09-15 22:15:13
515,8133,2021-09-16,外資及陸資,209089,67311,233130076,355387,225788,73716,247655200,...,312185,311057,52047,206501603,401438,-243342,-24488,-122866864,-89253,2021-09-16 22:59:03


In [29]:
last_investment = df_investment["FutureOINetQty"][-6:-1]
print(last_investment)
print(df_investment["FutureOINetQty"][-1:])
# print(last_investment.mean())

500   -226421
503   -228360
506   -226640
509   -246599
512   -243636
Name: FutureOINetQty, dtype: int64
515   -243342
Name: FutureOINetQty, dtype: int64


In [30]:
# signal_3: 外資期貨留倉是否空單還是多單
# 最近一次交易日與前一次做比較
# True:多單；False：空單

signal_3 = True if last_investment.iloc[-1] > 0 else False
signal_3

False

In [31]:
# signal_4: 外資期貨留倉數量是否增加
# 最近一次交易日與前一次的期貨留倉數量做比較
# True:增加；False：減少

signal_4 = True if last_investment.iloc[-1] >= last_investment.iloc[-2] else False
signal_4


True

In [32]:
# signal_5: 外資期貨留倉數量變化多少
# 最近一次交易日與前一次的期貨留倉數量做相減
# 顯示為變化數量

signal_5 = last_investment.iloc[-1] - last_investment.iloc[-2]
signal_5


2963

In [34]:
# signal_6: 外資期貨留倉數變動比例
# 為小數顯示
signal_6 = (last_investment.iloc[-1] - last_investment.iloc[-2])/last_investment.iloc[-2]
signal_6

-0.012015458294640286